# Montgomery Potential of Southern Ocean

Calculating interface heights between density surfaces and then the Montgomery Potential (layered model equivalent to pressure) anomaly from a reference Montgomery Potential of the 10 year time mean interface depths.

Takes about 30 mins per month, did it for 10 years as of 17/7/21. Doesn't need 48 cores of CPU but does need a lot of memory.

In [1]:
%matplotlib inline

import cosima_cookbook as cc
import matplotlib.pyplot as plt
import numpy as np
import netCDF4 as nc
import xarray as xr
import cftime
import glob
import dask
import os
import datetime
import matplotlib.colors as col
import dask.array as dsa
from cosima_cookbook import distributed as ccd
import cartopy.crs as ccrs
import cmocean as cm
from scipy.ndimage.filters import uniform_filter1d

import logging
logging.captureWarnings(True)
logging.getLogger('py.warnings').setLevel(logging.ERROR)

from dask.distributed import Client

### Start up dask cluster and cosima cookbook session

In [2]:
client = Client()
client

Client Scheduler: tcp://127.0.0.1:33095 Dashboard: /proxy/8787/status,Cluster Workers: 7 Cores: 28 Memory: 112.00 GiB


In [3]:
session = cc.database.create_session('/g/data/ik11/databases/cosima_master.db')

### Define what month and year you want

In [4]:
month = '02'
year = '2170'

### Define some slicing parameters

In [5]:
expt = '01deg_jra55v13_ryf9091'

time_slice= year + '-' + month
start_time = year+'-01-01'
end_time = year+'12-31'
# reference density value:
rho_0 = 1035.0
g = 9.81

#latitude range of Southern Ocean
## FULL SO ##
lat_range = slice(-70,-34.99)
lat_range_big =  slice(-70.05,-34.90)


### Load one month of saved binned daily isopycnal layer thickness data

In [8]:
%%time

saved_dir = '/g/data/x77/cy8964/Post_Process/New_SO/SO_saving_energy_terms_'
chunky = {"time":1,"isopycnal_bins":5,"yt_ocean":218,"xt_ocean":400 }
#month = '01'
h_binned = xr.open_dataset(saved_dir+'h_binned_'+year+'-'+month+'_test.nc', chunks = chunky).h_binned

# to scale up to a whole year:

# for i in np.arange(2,13):
#     month = str(i).zfill(2)
#     h_binned_i = xr.open_dataset(saved_dir+'h_binned_'+year+'-'+month+'.nc', chunks = chunky)
#     h_binned_i = h_binned_i.h_binned

#     h_binned = xr.concat([h_binned,h_binned_i], dim = 'time')



CPU times: user 39.1 ms, sys: 0 ns, total: 39.1 ms
Wall time: 80 ms


### Load one month of cosima cookbook daily eta_t

In [9]:
%%time
# Load eta_t for given year
eta_t = cc.querying.getvar(expt,'eta_t',session,ncfile='%daily%',start_time=start_time, end_time=end_time)
eta_t = eta_t.sel(yt_ocean=lat_range, time=time_slice)#slice(start_time,end_time))

CPU times: user 578 ms, sys: 0 ns, total: 578 ms
Wall time: 6.46 s


### Load saved long term (10 year) means of sea surface height and layer thickness

In [12]:
%%time
ssh = cc.querying.getvar(expt,'eta_t',session,ncfile='%daily%',start_time='2170-01-01', end_time='2179-12-31')
ssh = ssh.sel(yt_ocean=lat_range, time=slice('2170-01-01','2179-12-31'))

CPU times: user 632 ms, sys: 81.2 ms, total: 713 ms
Wall time: 1.47 s


%%time
ssh_avg = ssh.mean('time')
ssh_avg = ccd.compute_by_block(ssh_avg)
save_dir = '/g/data/x77/cy8964/Post_Process/New_SO/'
ds = xr.Dataset({'eta_t': ssh_avg})
ds.to_netcdf(save_dir+'ssh_avg_10yr.nc')

In [11]:
%%time
# Load saved h_bar and ssh_avg

chunky2 = {"isopycnal_bins":5,"yt_ocean":218,"xt_ocean":400 }
saved_dir2 = '/g/data/x77/cy8964/Post_Process/New_SO/'
h_binned_mean = xr.open_dataset(saved_dir2+'h_bar_10yr.nc', chunks = chunky2).h_bar
ssh_avg = xr.open_dataset(saved_dir2+'ssh_avg_10yr.nc',chunks = {"yt_ocean":218,"xt_ocean":400 }).ssh_avg




CPU times: user 37.8 ms, sys: 0 ns, total: 37.8 ms
Wall time: 99.2 ms


### Define g prime values for Montgomery potential calculation

In [12]:
%%time
## define isopycnal bins   
isopycnal_bins_sigma1 = 1000+ np.array([1,28,29,30,31,31.5,31.9,32,32.1,32.2,32.25,32.3,
                                        32.35,32.4,32.42,32.44,32.46,32.48,32.50,32.51,
                                        32.52,32.53,32.54,32.55,32.56,32.58,32.6,32.8,33,34,45])
## since bins are edges of bins, define middle densities to be used in 
middle_density = (isopycnal_bins_sigma1+ np.roll(isopycnal_bins_sigma1, -1))/2
middle_density[0]=1028.5
middle_density[-2]=1034.5
middle_density[-1]=1035

## define g primes - set g_prime[0] to be g
g_prime = g*(middle_density - np.roll(middle_density, 1))/rho_0
g_prime[0]=g
g_prime = xr.DataArray(g_prime, coords = [isopycnal_bins_sigma1], dims = ['isopycnal_bins'], name = 'g_prime')
                                             



CPU times: user 1.41 ms, sys: 0 ns, total: 1.41 ms
Wall time: 1.24 ms


### Find reference depth for each interface, long term mean (this only has to happen once)

For top layer (index isopycnal_bins = 0), depth is ssh_avg. For other layers, cumulatively sum the thickness of the layers above that interface. Do this by rolling so that thickness of layer 0 is in entry of layer 1, select isopycnal bins 1 and onwards with sel, then cumulatively sum.

In [13]:
%%time
#average top of interface heights (.roll is so that all h's get summed from top density bin, otherwise would miss top density bin's h value)
eta_ref = h_binned_mean.roll(isopycnal_bins = 1,roll_coords = False ).sel(isopycnal_bins = slice(1020,1050)).cumsum('isopycnal_bins')
eta_ref_0 = xr.zeros_like(h_binned_mean.isel(isopycnal_bins=0))
eta_ref = xr.concat([eta_ref_0,eta_ref], dim = 'isopycnal_bins')
eta_ref = eta_ref+ssh_avg

CPU times: user 50.7 ms, sys: 0 ns, total: 50.7 ms
Wall time: 46.4 ms


In [14]:
%%time
eta_ref = ccd.compute_by_block(eta_ref)

  0%|          | 0/216 [00:00<?, ?it/s]

CPU times: user 16.6 s, sys: 1.35 s, total: 18 s
Wall time: 42 s


In [15]:
%%time

save_dir = '/g/data/x77/cy8964/Post_Process/New_SO/'
ds = xr.Dataset({'eta_ref': eta_ref})
ds.to_netcdf(save_dir+'eta_ref_10yr.nc')

CPU times: user 224 ms, sys: 362 ms, total: 586 ms
Wall time: 607 ms


In [22]:
%xdel eta_ref
%xdel h_binned_mean
%xdel ssh_avg

In [23]:
%%time


eta_ref = xr.open_dataset('/g/data/x77/cy8964/Post_Process/SO/eta_ref_10yr.nc', 
                           chunks = {'isopycnal;_bins':5,'yt_ocean':218,'xt_ocean':400}).eta_ref

CPU times: user 17.6 ms, sys: 2.72 ms, total: 20.3 ms
Wall time: 18.5 ms


### Find depth as a function of time for each interface, then subtract reference depth to find interface anomaly etas
As before, but these have a time dimension and the subtraction of the long term mean.

In [16]:
%%time
# deviation from top of interface height from average
etas = h_binned.roll(isopycnal_bins = 1,roll_coords = False ).sel(isopycnal_bins = slice(1020,1050)).cumsum('isopycnal_bins')
etas_0 = xr.zeros_like(h_binned.isel(isopycnal_bins=0))
etas = xr.concat([etas_0,etas], dim = 'isopycnal_bins')
etas = etas+eta_t-eta_ref

CPU times: user 298 ms, sys: 19 ms, total: 317 ms
Wall time: 313 ms


### For some reason this is very intensive. Hence use ccd.compute_by_block so that memory usage is more controlled. Still takes a while.

In [17]:
%%time
etas = ccd.compute_by_block(etas)

  0%|          | 0/9408 [00:00<?, ?it/s]

CPU times: user 16min 3s, sys: 51.8 s, total: 16min 55s
Wall time: 27min 35s


### Save the etas as a file and then delete it and reopen it.

In [18]:
%%time

save_dir = '/g/data/x77/cy8964/Post_Process/SO/'
ds = xr.Dataset({'etas': etas})
ds.to_netcdf(save_dir+'etas_'+year+'-'+month+'.nc')



CPU times: user 47.5 ms, sys: 10.4 s, total: 10.5 s
Wall time: 19 s


In [19]:
%%time
%xdel etas
%xdel h_binned
%xdel eta_ref
%xdel h_binned_mean
%xdel ssh_avg

etas = xr.open_dataset('/g/data/x77/cy8964/Post_Process/SO/etas_'+year+'-'+month+'.nc', 
                           chunks = {'time':1, 'isopycnal;_bins':5,'yt_ocean':218,'xt_ocean':400}).etas



NameError: name 'h_binned_mean' is not defined
NameError: name 'ssh_avg' is not defined
CPU times: user 22.7 ms, sys: 1.42 ms, total: 24.2 ms
Wall time: 25.3 ms


### Calculate the Montgomery Potential anomaly, phi

For layer 0, this is g * etas

For layers below, it is g prime * etas + MP_anomaly above. I set the first entry of g_prime to be g so I can just multiply by g_prime for all layers

In [20]:
%%time

# compute MP anomaly
phi = rho_0*g_prime*etas
phi = phi.cumsum('isopycnal_bins')



CPU times: user 20.1 ms, sys: 2 ms, total: 22.1 ms
Wall time: 21.9 ms


In [21]:
%%time
phi = ccd.compute_by_block(phi)

  0%|          | 0/756 [00:00<?, ?it/s]

CPU times: user 48.6 s, sys: 15.2 s, total: 1min 3s
Wall time: 1min 49s


In [22]:
%%time

save_dir = '/g/data/x77/cy8964/Post_Process/SO/'
ds = xr.Dataset({'phi': phi})
ds.to_netcdf(save_dir+'Montgomery_Potential_Anomaly_'+year+'-'+month+'.nc')

CPU times: user 42.9 ms, sys: 10 s, total: 10.1 s
Wall time: 15.3 s
